In [ ]:
import numpy as np
import faiss
import psycopg2
import time
from psycopg2.extras import execute_values
import pandas as pd
from typing import Tuple, List

def create_random_vectors(num_vectors=100_000, dim=4096):
    vectors = np.random.randn(num_vectors, dim).astype('float32')
    vectors = vectors / np.linalg.norm(vectors, axis=1)[:, np.newaxis]
    return vectors

def setup_pgvector_db():
    conn = psycopg2.connect("dbname=postgres")
    conn.autocommit = True
    cur = conn.cursor()
    
    cur.execute("DROP DATABASE IF EXISTS vector_benchmark;")
    cur.execute("CREATE DATABASE vector_benchmark;")
    cur.close()
    conn.close()
    
    conn = psycopg2.connect("""
        dbname=vector_benchmark 
        options='-c maintenance_work_mem=2GB 
                -c synchronous_commit=off '
    """)
    
    cur = conn.cursor()
    cur.execute("SET work_mem = '1GB';")
    cur.execute("SET maintenance_work_mem = '2GB';")
    cur.execute("CREATE EXTENSION IF NOT EXISTS vector;")
    
    cur.execute("""
        CREATE TABLE items (
            id SERIAL PRIMARY KEY,
            embedding vector(4096)
        );
    """)
    
    cur.execute("ALTER TABLE items SET (autovacuum_enabled = false);")
    conn.commit()
    return conn, cur

def fetch_vectors_from_db(cur) -> np.ndarray:
    """Fetch all vectors from database efficiently"""
    print("Fetching vectors from database...")
    start_time = time.time()
    
    # Fetch in batches to manage memory
    batch_size = 10000
    vectors = []
    
    cur.execute("SELECT COUNT(*) FROM items")
    total_rows = cur.fetchone()[0]
    
    for offset in range(0, total_rows, batch_size):
        cur.execute(
            "SELECT embedding FROM items ORDER BY id LIMIT %s OFFSET %s",
            (batch_size, offset)
        )
        batch = cur.fetchall()
        vectors.extend([np.array(row[0]) for row in batch])
    
    vectors_array = np.vstack(vectors).astype('float32')
    
    print(f"Fetching vectors took {time.time() - start_time:.2f} seconds")
    return vectors_array

def insert_vectors_to_postgres(vectors, cur, conn, batch_size=5000):
    print("Inserting vectors to PostgreSQL...")
    start_time = time.time()
    
    data = [(vector.tolist(),) for vector in vectors]
    cur.execute("BEGIN;")
    
    execute_values(
        cur,
        "INSERT INTO items (embedding) VALUES %s",
        data,
        template="(%s::vector)",
        page_size=batch_size
    )
    
    conn.commit()
    cur.execute("ALTER TABLE items SET (autovacuum_enabled = true);")
    conn.commit()
    
    print(f"Insertion took {time.time() - start_time:.2f} seconds")

def create_pgvector_index(cur, conn):
    print("Creating pgvector index...")
    start_time = time.time()
    
    cur.execute("SET maintenance_work_mem = '2GB';")
    cur.execute("""
        CREATE INDEX ON items USING ivfflat (embedding vector_cosine_ops) 
        WITH (lists = 1000);
    """)
    conn.commit()
    print(f"Index creation took {time.time() - start_time:.2f} seconds")

def benchmark_pgvector(vectors, cur, num_queries=100, k=20) -> Tuple[pd.Series, float]:
    print(f"\nTesting pgvector with {num_queries} queries...")
    query_vectors = vectors[:num_queries]
    
    times = []
    start_total = time.time()
    
    for query in query_vectors:
        start = time.time()
        cur.execute("""
            SELECT id, embedding <=> %s as distance 
            FROM items 
            ORDER BY embedding <=> %s 
            LIMIT %s
        """, (query.tolist(), query.tolist(), k))
        results = cur.fetchall()
        times.append(time.time() - start)
    
    total_time = time.time() - start_total
    return pd.Series(times).describe(), total_time

def benchmark_faiss_with_loading(cur, num_queries=100, k=20) -> Tuple[pd.Series, float, dict]:
    print(f"\nTesting FAISS (including data loading) with {num_queries} queries...")
    timings = {}
    
    # Time vector loading
    start_load = time.time()
    vectors = fetch_vectors_from_db(cur)
    load_time = time.time() - start_load
    timings['load_time'] = load_time
    
    # Time index building
    start_build = time.time()
    index = faiss.IndexFlatL2(vectors.shape[1])
    index.add(vectors)
    build_time = time.time() - start_build
    timings['build_time'] = build_time
    
    # Time queries
    query_vectors = vectors[:num_queries]
    times = []
    start_total = time.time()
    
    for query in query_vectors:
        start = time.time()
        D, I = index.search(query.reshape(1, -1), k)
        times.append(time.time() - start)
    
    query_time = time.time() - start_total
    timings['total_query_time'] = query_time
    
    return pd.Series(times).describe(), sum(timings.values()), timings

def run_benchmark():
    print("Generating random vectors...")
    vectors = create_random_vectors()
    
    # Setup and populate PostgreSQL
    conn, cur = setup_pgvector_db()
    insert_vectors_to_postgres(vectors, cur, conn)
    create_pgvector_index(cur, conn)
    
    # Run benchmarks
    pg_stats, pg_total = benchmark_pgvector(vectors, cur)
    faiss_stats, faiss_total, faiss_timings = benchmark_faiss_with_loading(cur)
    
    print("\n=== Results ===")
    print("\npgvector query times (seconds):")
    print(pg_stats)
    print(f"Total time for pgvector: {pg_total:.2f} seconds")
    
    print("\nFAISS timings (seconds):")
    print("Loading data:", faiss_timings['load_time'])
    print("Building index:", faiss_timings['build_time'])
    print("Query execution:", faiss_timings['total_query_time'])
    print("\nFAISS query statistics (seconds):")
    print(faiss_stats)
    print(f"Total time for FAISS (including all overhead): {faiss_total:.2f} seconds")
    
    # Cleanup
    cur.execute("DROP INDEX items_embedding_idx;")
    cur.close()
    conn.close()
    
    conn = psycopg2.connect("dbname=postgres")
    conn.autocommit = True
    cur = conn.cursor()
    cur.execute("DROP DATABASE vector_benchmark;")
    cur.close()
    conn.close()

In [ ]:
run_benchmark()